In [3]:
place = 'los angeles'
keyword = 'android developer'

In [4]:
def get_job_description(job_url: str) -> str:
    import requests
    from bs4 import BeautifulSoup
    
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }
    job_description = ''
    response = requests.post(job_url, headers=headers)
    if response.status_code != 200:
        return (job_description, 0)
    result = BeautifulSoup(response.content,'lxml')
    if not result.find('div', class_='jobDesc'):
        return (job_description, 1)
    return (result.find('div', class_='jobDesc').get_text(), 0)

In [5]:
def get_job_list(place: str, keyword: str) -> list:
    import requests
    from bs4 import BeautifulSoup
    import time
    
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }
    
    location_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.01',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }

    data = {"term": place,
            "maxLocationsToReturn": 10}
    
    location_url = "https://www.glassdoor.co.in/findPopularLocationAjax.htm?"
    
    try:
        # get location id for search location
        location_response = requests.post(location_url, headers=location_headers, data=data).json()
        place_id = location_response[0]['locationId']
        job_list_url = 'https://www.glassdoor.com/Job/jobs.htm'
        absolute_url = 'https://www.glassdoor.com'
        data = {
            'clickSource': 'searchBtn',
            'sc.keyword': keyword,
            'locT': 'C',
            'locId': place_id,
            'jobType': ''
        }
        
        result_list = list()
        
        if place_id:
            for i in range(30):     # don't know why, the website can only show the first 30 pages
                response = requests.post(job_list_url, headers=headers, data=data)
                results_page = BeautifulSoup(response.content,'lxml')
                jobs = results_page.find('', class_='jlGrid hover').find_all('li')
                
                for job in jobs:
                    # get the attributes of the job
                    job_name = job.find('div', class_='flexbox jobTitle').find('a').get_text()
                    job_url = absolute_url+job.find('div', class_='flexbox jobTitle').find('a').get('href')
                    company_location = job.find('div', class_='flexbox empLoc').find('div').get_text().split(' – ')
                    job_company = company_location[0][1:]
                    job_city = company_location[1].split(', ')[0]
                    job_state = company_location[1].split(', ')[1]
                    if job.find('span', class_='green small'):
                        job_salary = job.find('span', class_='green small').get_text()[1:]
                    else:
                        job_salary = ''
                    print(job_url)
                    job_description, status = get_job_description(job_url)
                    if status == 1:
                        return result_list
                    result_list.append((job_name, job_company, job_city, job_state, job_salary, job_description, job_url))
                    
                if not results_page.find('li', class_='next'):
                    break   # only one page    
                if not results_page.find('li', class_='next').find('a'):
                    break   # reach the end page
                # next page
                job_list_url = absolute_url + results_page.find('li', class_='next').find('a').get('href')
                time.sleep(2)
                    
            return result_list         
        else:
            print('Location not available.')
              
    except:
        print('Fail to find jobs.')

In [6]:
result = get_job_list(place, keyword)

https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=322198&s=149&guid=000001675dcee030a4cb8c2fc2fbaebd&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_077ff557&cb=1543467098667&jobListingId=2909234457
https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=441344&s=149&guid=000001675dcee030a4cb8c2fc2fbaebd&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_1cd59d39&cb=1543467098669&jobListingId=2945204971
https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=318376&s=58&guid=000001675dcee030a03c546b683127e5&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_106b12a0&cb=1543467098670&jobListingId=2960185184
https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=85058&s=58&guid=000001675dcee030a03c546b683127e5&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_4d10fa09&cb=1543467098671&jobListingId=3009314176
https://www.glassdoor.com/partner/jobListing.htm?pos=103&ao=148364&

https://www.glassdoor.com/partner/jobListing.htm?pos=207&ao=295315&s=58&guid=000001675dcf7e16906c0fdf8e431979&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&aa=1&ea=1&cs=1_1f4801ae&cb=1543467139051&jobListingId=2993879561
https://www.glassdoor.com/partner/jobListing.htm?pos=208&ao=428589&s=58&guid=000001675dcf7e16906c0fdf8e431979&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_5641d0e1&cb=1543467139052&jobListingId=3027112407
https://www.glassdoor.com/partner/jobListing.htm?pos=209&ao=9619&s=58&guid=000001675dcf7e16906c0fdf8e431979&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_1c01b4f8&cb=1543467139053&jobListingId=2919320170
https://www.glassdoor.com/partner/jobListing.htm?pos=210&ao=349475&s=58&guid=000001675dcf7e16906c0fdf8e431979&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_988bf4e6&cb=1543467139054&jobListingId=2781569200
https://www.glassdoor.com/partner/jobListing.htm?pos=211&ao=4285

https://www.glassdoor.com/partner/jobListing.htm?pos=312&ao=4120&s=58&guid=000001675dd017fdbf19f09b5dcdf041&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_3e84bcdb&cb=1543467178435&jobListingId=3040439808
https://www.glassdoor.com/partner/jobListing.htm?pos=313&ao=395376&s=58&guid=000001675dd017fdbf19f09b5dcdf041&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_d044cd18&cb=1543467178436&jobListingId=2950981837
https://www.glassdoor.com/partner/jobListing.htm?pos=314&ao=85058&s=58&guid=000001675dd017fdbf19f09b5dcdf041&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_9f829f0c&cb=1543467178437&jobListingId=3014543935
https://www.glassdoor.com/partner/jobListing.htm?pos=315&ao=389273&s=58&guid=000001675dd017fdbf19f09b5dcdf041&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_b213fd76&cb=1543467178438&jobListingId=2943179956
https://www.glassdoor.com/partner/jobListing.htm?pos=316&ao=288527&s=58&gui

https://www.glassdoor.com/partner/jobListing.htm?pos=418&ao=291820&s=58&guid=000001675dd0c85da5205475fa411c2a&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_5b1e5de0&cb=1543467223605&jobListingId=3040612037
https://www.glassdoor.com/partner/jobListing.htm?pos=419&ao=160631&s=58&guid=000001675dd0c85da5205475fa411c2a&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_a27f3b31&cb=1543467223606&jobListingId=2907045221
https://www.glassdoor.com/partner/jobListing.htm?pos=420&ao=242359&s=58&guid=000001675dd0c85da5205475fa411c2a&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_ffd1b306&cb=1543467223607&jobListingId=3033563995
https://www.glassdoor.com/partner/jobListing.htm?pos=421&ao=352789&s=58&guid=000001675dd0c85da5205475fa411c2a&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_d5edfc3a&cb=1543467223608&jobListingId=3017432501
https://www.glassdoor.com/partner/jobListing.htm?pos=422&ao=182894&s=58&

https://www.glassdoor.com/partner/jobListing.htm?pos=523&ao=8095&s=58&guid=000001675dd1620dad7b97317c2c7933&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_53e0eb00&cb=1543467262945&jobListingId=2358397286
https://www.glassdoor.com/partner/jobListing.htm?pos=524&ao=291820&s=58&guid=000001675dd1620dad7b97317c2c7933&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_ae41dde6&cb=1543467262947&jobListingId=3039804328
https://www.glassdoor.com/partner/jobListing.htm?pos=525&ao=135475&s=58&guid=000001675dd1620dad7b97317c2c7933&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_851ba70a&cb=1543467262948&jobListingId=3040577359
https://www.glassdoor.com/partner/jobListing.htm?pos=526&ao=160415&s=58&guid=000001675dd1620dad7b97317c2c7933&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_622ace5b&cb=1543467262949&jobListingId=2886791012
https://www.glassdoor.com/partner/jobListing.htm?pos=527&ao=430909&s=58&gu

...

https://www.glassdoor.com/partner/jobListing.htm?pos=2529&ao=242359&s=58&guid=000001675dde5e21be87dcfd55540b6d&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_d40ee4f0&cb=1543468113955&jobListingId=3033503215
https://www.glassdoor.com/partner/jobListing.htm?pos=2530&ao=352789&s=58&guid=000001675dde5e21be87dcfd55540b6d&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_431df208&cb=1543468113956&jobListingId=3018383290
https://www.glassdoor.com/partner/jobListing.htm?pos=2601&ao=322017&s=58&guid=000001675ddf0e9c96d541ede7661888&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_7f9600ef&cb=1543468159093&jobListingId=3039934865
https://www.glassdoor.com/partner/jobListing.htm?pos=2602&ao=133050&s=58&guid=000001675ddf0e9c96d541ede7661888&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_fbb2fac1&cb=1543468159094&jobListingId=3027741234
https://www.glassdoor.com/partner/jobListing.htm?pos=2603&ao=160631&

https://www.glassdoor.com/partner/jobListing.htm?pos=2705&ao=4120&s=58&guid=000001675ddfb4d4a5e9fe15e4498cd7&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&aa=1&ea=1&cs=1_4eb90471&cb=1543468201732&jobListingId=2576708695
https://www.glassdoor.com/partner/jobListing.htm?pos=2706&ao=4120&s=58&guid=000001675ddfb4d4a5e9fe15e4498cd7&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&aa=1&ea=1&cs=1_5259d28d&cb=1543468201734&jobListingId=2576708650
https://www.glassdoor.com/partner/jobListing.htm?pos=2707&ao=4128&s=58&guid=000001675ddfb4d4a5e9fe15e4498cd7&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_aec23ca3&cb=1543468201735&jobListingId=3020215701
https://www.glassdoor.com/partner/jobListing.htm?pos=2708&ao=283198&s=58&guid=000001675ddfb4d4a5e9fe15e4498cd7&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_665fbbc6&cb=1543468201736&jobListingId=3025469455
https://www.glassdoor.com/partner/jobListing.htm?pos=2

https://www.glassdoor.com/partner/jobListing.htm?pos=2811&ao=352789&s=58&guid=000001675de05899b347710e0b4c2246&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_31613a33&cb=1543468243575&jobListingId=3040462575
https://www.glassdoor.com/partner/jobListing.htm?pos=2812&ao=242359&s=58&guid=000001675de05899b347710e0b4c2246&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_5196415c&cb=1543468243576&jobListingId=3021330156
https://www.glassdoor.com/partner/jobListing.htm?pos=2813&ao=3839&s=58&guid=000001675de05899b347710e0b4c2246&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_76869c32&cb=1543468243577&jobListingId=3008767010
https://www.glassdoor.com/partner/jobListing.htm?pos=2814&ao=4120&s=58&guid=000001675de05899b347710e0b4c2246&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_e35a8308&cb=1543468243578&jobListingId=3039931514
https://www.glassdoor.com/partner/jobListing.htm?pos=2815&ao=4120&s=58&g

https://www.glassdoor.com/partner/jobListing.htm?pos=2917&ao=352789&s=58&guid=000001675de0fc0cb5e5723672a944ac&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_221ec61b&cb=1543468285567&jobListingId=3033949596
https://www.glassdoor.com/partner/jobListing.htm?pos=2918&ao=280982&s=58&guid=000001675de0fc0cb5e5723672a944ac&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_64b78503&cb=1543468285568&jobListingId=3033236573
https://www.glassdoor.com/partner/jobListing.htm?pos=2919&ao=352789&s=58&guid=000001675de0fc0cb5e5723672a944ac&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_f21d1954&cb=1543468285569&jobListingId=3017273062
https://www.glassdoor.com/partner/jobListing.htm?pos=2920&ao=328443&s=58&guid=000001675de0fc0cb5e5723672a944ac&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_a294792e&cb=1543468285570&jobListingId=2918470011
https://www.glassdoor.com/partner/jobListing.htm?pos=2921&ao=4120&s=

https://www.glassdoor.com/partner/jobListing.htm?pos=3023&ao=352789&s=58&guid=000001675de19b81ad4a24e6ee9b0065&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_0419649d&cb=1543468326192&jobListingId=3033949596
https://www.glassdoor.com/partner/jobListing.htm?pos=3024&ao=242359&s=58&guid=000001675de19b81ad4a24e6ee9b0065&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_602120b0&cb=1543468326193&jobListingId=3027935841
https://www.glassdoor.com/partner/jobListing.htm?pos=3025&ao=85058&s=58&guid=000001675de19b81ad4a24e6ee9b0065&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_3953bf3d&cb=1543468326194&jobListingId=3027615037
https://www.glassdoor.com/partner/jobListing.htm?pos=3026&ao=58811&s=58&guid=000001675de19b81ad4a24e6ee9b0065&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_b624a2dd&cb=1543468326195&jobListingId=3000709982
https://www.glassdoor.com/partner/jobListing.htm?pos=3027&ao=352789&s=